In [18]:
# CELDA: Consolidación OpenPhish (sin filtros de España)
# Lee todos los CSV de data/raw/phishing/openphish/, normaliza columnas básicas,
# añade trazabilidad y guarda en data/processed/openphish_consolidado.csv

import pandas as pd
from pathlib import Path
from datetime import datetime
import glob, os

# 1) Asegurar CWD en raíz del repo (si ejecutas desde notebooks/)
if Path.cwd().name == "notebooks" and Path("../data").exists():
    %cd ..

SRC_DIR = Path("data/raw/phishing/openphish")
OUT_DIR = Path("data/processed"); OUT_DIR.mkdir(parents=True, exist_ok=True)
OUT_FILE = OUT_DIR / "openphish_consolidado.csv"

# 2) Localizar ficheros
rutas = sorted(glob.glob(str(SRC_DIR / "*.csv")))
assert rutas, f"No hay CSV en {SRC_DIR.resolve()}"
print(f"Ficheros detectados: {len(rutas)}")

# 3) Cargar y normalizar mínimo
frames = []
total_leidas = 0

for p in rutas:
    df = pd.read_csv(p)

    total_leidas += len(df)
    lower = {c.lower(): c for c in df.columns}

    # Columna URL obligatoria
    if "url" not in lower:
        raise ValueError(f"{p} sin columna 'url'. Columnas: {df.columns.tolist()}")
    url_col = lower["url"]

    # Normalización: url en string, strip, solo http(s)
    df["url"] = df[url_col].astype(str).str.strip()
    df = df[df["url"].str.startswith(("http://","https://"), na=False)].copy()

    # Trazabilidad mínima
    df["fuente"] = df.get("fuente", "OpenPhish")
    df["fecha_hora_recoleccion"] = df.get(
        "fecha_hora_recoleccion",
        datetime.utcnow().isoformat(timespec="seconds") + "Z"
    )
    df["__source_file"] = os.path.basename(p)

    frames.append(df[["url","fuente","fecha_hora_recoleccion","__source_file"]])

# 4) Consolidar y de-duplicar por URL
consol = pd.concat(frames, ignore_index=True)
antes = len(consol)
consol = consol.drop_duplicates(subset=["url"]).reset_index(drop=True)
eliminados = antes - len(consol)

# 5) Guardar
consol.to_csv(OUT_FILE, index=False)

print("✅ Consolidación OpenPhish completada")
print(f" - Filas leídas totales: {total_leidas}")
print(f" - Únicas por URL: {len(consol)} (duplicados eliminados: {eliminados})")
print(f" - Guardado en: {OUT_FILE}")
consol.head(5)


Ficheros detectados: 15
✅ Consolidación OpenPhish completada
 - Filas leídas totales: 4500
 - Únicas por URL: 3863 (duplicados eliminados: 637)
 - Guardado en: data/processed/openphish_consolidado.csv


,url,fuente,fecha_hora_recoleccion,__source_file
0,https://monday.kimberlywoodrich.workers.dev/,OpenPhish,2025-08-18 17:16:36,2025-08-18.csv
1,https://mybdoonline.device-personal.workers.de...,OpenPhish,2025-08-18 17:16:36,2025-08-18.csv
2,https://bruker.directsmailsolution.com/qrc-ac/...,OpenPhish,2025-08-18 17:16:36,2025-08-18.csv
3,https://avisou.click/x,OpenPhish,2025-08-18 17:16:36,2025-08-18.csv
4,https://bdo-app.sbdoonline.workers.dev/bdo-for...,OpenPhish,2025-08-18 17:16:36,2025-08-18.csv
